In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('Reduction-Operations')
sc = SparkContext(conf = conf)

24/04/19 17:58:57 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 10.90.9.140 instead (on interface en0)
24/04/19 17:58:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/19 17:58:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
lines = sc.parallelize(["hello world", "hi"])
words_map = lines.map(lambda line : line.split(" "))
words_fmap = lines.flatMap(lambda line : line.split(" "))

In [4]:
words_map.collect()

[['hello', 'world'], ['hi']]

In [5]:
words_fmap.collect()

['hello', 'world', 'hi']

24/04/19 18:24:21 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 169322 ms exceeds timeout 120000 ms
24/04/19 18:24:21 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/19 18:24:24 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc